In [1]:
from pathlib import Path
import numpy as np
import datetime
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt
import pyproj
import cartopy.feature as cfe
import cartopy.crs as ccrs
import pandas as pd
from pyproj import Transformer
from pyproj import CRS
import shapefile as shp
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import fiona 
import rioxarray
import geopandas as gpd
from shapely.geometry import mapping
import geoplot
from rasterio import features
import matplotlib.colors as colors
from numpy import ma

C:\Users\Pearl\anaconda3\envs\dissenv\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


# Functions

In [2]:
def plot_nearby_ice_shelves(shapefile,x_lims,y_lims):
    """
    Input: 
    shapefile -> shape file (Geopandas Dataframe format),
    x_lims -> minimum longitude and maximum longitude to search for ice shelves (Tuple)
    y_lims -> minimum latitude and maximum latitude to search for ice shelves (Tuple)
    
    Output:
    Returns a masked Geopandas Dataframe of ice shelves within the x_lims, y_lims defined
    Plots Graph with reference numbers
    """
    border = [(x_lims[0],y_lims[0]), (x_lims[1],y_lims[0]), (x_lims[1],y_lims[1]),(x_lims[0],y_lims[1])]
    bounds_poly=Polygon(shell=border)
    mask = shapefile['geometry'].apply(lambda x: bounds_poly.contains(x))
    within_bounds = (shapefile[mask])
    ax = within_bounds.plot(figsize=(12, 12))
    within_bounds.apply(lambda x: ax.annotate(text=x['recnum'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);
    plt.show()
    return within_bounds

def read_file_paths(filename):
    this_dir=Path.cwd()
    data_dir=this_dir.parent/Path("Data")
    files = data_dir.rglob(filename)
    for i in files:
        data = i
    return data

def one_ice_shelf_only_mask(all_is,rn):
    """
    Input: 
    all_is -> Geopandas Dataframe of all ice shelves masked within a certain bound
    recnum-> the ref id needed for one particular ice_shelf
    Output: Geopandas Dataframe of just the one ice shelf defined by the record number rn
    """
    return all_is.loc[all_is.recnum==rn]

def ice_shelf_mask_rotate(ice_shelf,out_crs):
    """
    Input: 
    ice_shelf -> Geopandas Dataframe of just the one ice shelf
    out_crs-> the crs we want to convert the mask to
    Output: Geopandas Dataframe of just the one ice shelf defined by the record number rn
    """
    return ice_shelf.geometry.to_crs(data_crs)
    
def resample_summer(data,summer_months):
    """
    Input: 
    ds -> raw data set (XArray DataSet format),
    summer_months-> List for the years we want to filter
    Output:
    Returns a filtered XArray DataSet with only the summer months (Oct,Nov,Dec,Jan,Feb,Mar)
    """
    d2 = data.sel(time=data.time.dt.month.isin(summer_months))
    return d2

def reduce_spatial_data(ds,x_lims,y_lims,summer_months):
    """
    Input: 
    ds -> raw data set (XArray DataSet format),
    x_lims -> minimum longitude and maximum longitude to crop (Tuple)
    y_lims -> minimum latitude and maximum latitude to crop (Tuple)
    summer_months-> List for the years we want to filter
    
    Output:
    Returns a clipped XArray DataSet with only the summer months (Oct,Nov,Dec,Jan,Feb,Mar) and reduced space
    """
        
    summer_ds=resample_summer(ds,summer_months)
    minx,miny,maxx,maxy= transproj.transform_bounds(min(x_lims),min(y_lims),max(x_lims),max(y_lims))
    mask_lon = (summer_ds.rlon >= minx) & (summer_ds.rlon <= maxx)
    mask_lat = (summer_ds.rlat >= miny) & (summer_ds.rlat <= maxy)
    summer_ds= summer_ds.where(mask_lon & mask_lat, drop=True)
    return summer_ds

def masked_ds_only(ds_og,ds_masked,variable):
    """
    ds_og -> Original XArray Dataset that is masked
    ds_masked -> Xarray DataArray that is masked
    variable-> Variable to include (str)
    """
    ds_og=ds_og.drop_vars(variable)
    ds_og[variable] = ds_masked
    ds_og=ds_og.dropna(dim='rlat',how='all')
    ds_og=ds_og.dropna(dim='rlon',how='all')
    return ds_og

def resample_bounds(data,frequency):
    """
    resample data set to given frequency
    """
    d2=data.resample(time=frequency).mean()
    lat = d2['rlat'].values
    lon = d2['rlon'].values
    lons, lats = np.meshgrid(lon, lat)
    return d2,lons,lats

def mask_data(ds):
    ds['v10m']=ds.v10m.rio.set_spatial_dims(x_dim='rlon', y_dim='rlat', inplace=True)
    c=ds.v10m.rio.write_crs(data_crs).rio.clip(geometries=rotated_mask.geometry.apply(mapping),crs=data_crs,all_touched=False,drop=True,from_disk=True)
    ds= masked_ds_only(ds,c,'v10m')
    return ds

In [3]:
file2016='v10m.KNMI-2016.XPEN055.ERA5-3H_RACMO2.3p2.3H.nc'
file2021='v10m.KNMI-2021.XPEN055.ERA5-3H_RACMO2.3p2.3H.nc'
shape_name='my556bx0395.shx'

In [4]:
rd_2016=xr.open_dataset(read_file_paths(file2016)) 
rd_2021=xr.open_dataset(read_file_paths(file2021)) 
antarctica_shape = gpd.read_file(read_file_paths(shape_name))

OSError: [Errno -101] NetCDF: HDF error: b'D:\\Diss\\Data\\2016 Sample Data\\v10m.KNMI-2016.XPEN055.ERA5-3H_RACMO2.3p2.3H.nc'

In [ ]:
data_crs=CRS('-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=30.0 +o_lon_p=0')
from_lat_lon_crs = CRS.from_proj4("+proj=latlon")
transproj = Transformer.from_crs(from_lat_lon_crs, data_crs)

y_lim = (-75,-66) # latitude 
x_lim = (-78,-65) #longitude

In [ ]:
rd_2021

In [ ]:
within_bounds=plot_nearby_ice_shelves(antarctica_shape,x_lim,y_lim)
gviis_mask=one_ice_shelf_only_mask(within_bounds,0)
rotated_mask=ice_shelf_mask_rotate(gviis_mask,data_crs)

rd_2016=reduce_spatial_data(rd_2016,x_lim,y_lim,[1, 2, 3, 10, 11, 12])
rd_2016=rd_2016[['v10m']]

rd_2021=reduce_spatial_data(rd_2021,x_lim,y_lim,[1, 2, 3, 10, 11, 12])
rd_2021=rd_2021[['v10m']]

In [ ]:
rd_2016=mask_data(rd_2016)
rd_2021=mask_data(rd_2021)

In [ ]:
v10m_data = xr.concat([rd_2016, rd_2021],'time')

In [ ]:
v10m_data

In [ ]:
faceW = 79 #number of pixels in image width
faceH = 46 #number of pixels i|n image height
first_map = np.reshape(np.asarray(v10m_data.v10m.isel(time=1).values),(faceH,faceW)).T
plt.imshow(first_map)
plt.show()

In [ ]:
filename=v10m_data.attrs['title']+'.nc'
this_dir=Path.cwd()
out_path=this_dir.parent/Path("Data/Cleaned and masked NETCDF files"+'/'+filename)

In [ ]:
v10m_data.v10m.attrs

In [ ]:
del v10m_data.v10m.attrs['grid_mapping']
v10m_data.to_netcdf(out_path)